<a href="https://colab.research.google.com/github/JohnMommers/FILTER-MS-DATAFRAME/blob/main/FILTER_MS_DATAFRAME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FILTER MS DATAFRAME  
- filter on elements  
- filter on max number of elements  
- save new dataframe  


In [10]:
import pandas as pd
import re

In [4]:
# LOAD DATAFRAME
# Mass spectra: intensities of mz1 to mz1000 (column 4 to 1004)
# Smiles (=structures): last column
ms = pd.read_pickle('/content/drive/MyDrive/NIST/nist_08_Common_all_smiles_no_None.pkl')
ms.head()

,Name,Form,Mw,Cas,mz1,mz2,mz3,mz4,mz5,mz6,mz7,mz8,mz9,mz10,mz11,mz12,mz13,mz14,mz15,mz16,mz17,mz18,mz19,mz20,mz21,mz22,mz23,mz24,mz25,mz26,mz27,mz28,mz29,mz30,mz31,mz32,mz33,mz34,mz35,mz36,...,mz963,mz964,mz965,mz966,mz967,mz968,mz969,mz970,mz971,mz972,mz973,mz974,mz975,mz976,mz977,mz978,mz979,mz980,mz981,mz982,mz983,mz984,mz985,mz986,mz987,mz988,mz989,mz990,mz991,mz992,mz993,mz994,mz995,mz996,mz997,mz998,mz999,mz1000,Filter_Form,smiles
0,Ethyl 3-ethoxy-cis-crotonate,C8H14O3,158,5331-73-7,0,0,0,0,0,0,0,0,0,0,0,2,3,24,127,5,0,4,13,0,0,0,0,0,6,83,558,93,748,28,45,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,CCOC(/C)=C/C(=O)OCC
1,"Ethanone, 1-(3-methyloxiranyl)-",C5H8O2,100,17257-79-3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,40,110,85,170,10,15,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,CC1OC1C(C)=O
2,"2-Pentanone, 5-(1,2-propadienyloxy)-",C8H12O2,140,55702-69-7,0,0,0,0,0,0,0,0,0,0,0,1,4,19,110,2,1,3,0,0,0,0,0,0,5,41,117,29,74,2,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,CC(=O)CCCO[CH]=[C]=[CH2]
3,"n-Butyl (1,1-dimethylbutyl) sulfide",C10H22S,174,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,CCCCSC(C)(C)CCC
4,"Sulfurous acid, isohexyl 2-propyl ester",C9H20O3S,208,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,70,3,47,0,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,CC(C)CCCO[S](=O)OC(C)C


In [23]:
# This simple function converts a molecular formula into
# a dictionary containing: ELEMENT(str) : COUNT(int)
# C10H22O2Cl --> 'C':10, 'H':22, 'O':2, 'Cl':1

def form_to_dict(form_string):
  form = form_string
  x = re.findall('[A-Z][a-z]\d*|[A-Z]\d*',form)
  d = {'C':0, 'H':0, 'O':0, 'N':0, 'S':0}
  for i in x:
    element = re.search('[A-Z][a-z]|[A-Z]', i)
    count = re.search('[0-9]+',i)
    e = str(element.group())
    if count is not None:
      d[e] = int(count.group())
    else:
      d[e] = 1
  return d

In [36]:
filter = []
for i in ms['Form']:
  d = form_to_dict(i)
  if d['C']>0 and d['H']>0 and d['O']<=1 and d['N']<=1 and d['S']<=1 and len(d)==5:
    #print (i)
    filter.append(1)
  else:
    filter.append(0)

In [37]:
ms['Filter'] = filter

In [39]:
ms_filter = ms[ms['Filter']==1]

In [41]:
ms_filter

,Name,Form,Mw,Cas,mz1,mz2,mz3,mz4,mz5,mz6,mz7,mz8,mz9,mz10,mz11,mz12,mz13,mz14,mz15,mz16,mz17,mz18,mz19,mz20,mz21,mz22,mz23,mz24,mz25,mz26,mz27,mz28,mz29,mz30,mz31,mz32,mz33,mz34,mz35,mz36,...,mz964,mz965,mz966,mz967,mz968,mz969,mz970,mz971,mz972,mz973,mz974,mz975,mz976,mz977,mz978,mz979,mz980,mz981,mz982,mz983,mz984,mz985,mz986,mz987,mz988,mz989,mz990,mz991,mz992,mz993,mz994,mz995,mz996,mz997,mz998,mz999,mz1000,Filter_Form,smiles,Filter
3,"n-Butyl (1,1-dimethylbutyl) sulfide",C10H22S,174,NaN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,CCCCSC(C)(C)CCC,1
13,"Hexane, 2,3,3-trimethyl-",C9H20,128,16747-28-7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,158,19,126,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,CCCC(C)(C)C(C)C,1
14,"2-Pentanone, 3-ethyl-3-methyl-",C8H16O,128,19780-65-5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,0,0,0,0,0,0,0,0,0,7,67,10,65,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,CCC(C)(CC)C(C)=O,1
18,"Heptane, 3,4,5-trimethyl-",C10H22,142,20278-89-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,172,80,330,8,0,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,CCC(C)C(C)C(C)CC,1
19,3-Methyl-3-hexen-2-ol,C7H14O,114,76966-27-3,0,0,0,0,0,0,0,0,0,0,0,0,0,19,61,0,0,0,0,0,0,0,0,0,0,26,205,144,154,0,27,24,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,CCC=C(C)C(C)O,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205043,Hex-5-enylamine,C6H13N,99,34825-70-2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,113,123,65,999,55,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,NCCCCC=C,1
205044,"3-Amino-2,2-dimethyl-1-propanol",C5H13NO,103,26734-09-8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,61,1,0,0,0,0,0,0,4,48,71,80,999,79,13,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,CC(C)(CN)CO,1
205048,"Azocine, octahydro-",C7H15N,113,1121-92-2,0,0,0,0,0,0,0,0,0,0,0,0,0,3,33,1,0,0,0,0,0,0,0,0,0,19,161,242,158,999,28,16,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,C1CCCNCCC1,1
205051,N-n-Butylpropionamide,C7H15NO,129,2955-67-1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,10,0,1,4,0,0,0,0,0,0,0,13,122,68,399,999,15,7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,True,CCCCNC(=O)CC,1


In [44]:
ms_filter.to_csv('/content/drive/MyDrive/NIST/ms_only_CHO2N2S2', index=False)